# 제로베이스데이터사이언스 스쿨 원데이 프로젝트
> ## 세계 테러 데이터 분석 - SQL 사용하기

---

#### 총 8문제 100점 만점

- 문제 1 10점  
- 문제 2,3,5,6,7,8번 각 10점
- 문제 4번 30점
- 총점 100점

#### 유의사항

- 모든 정답 코드는 jupyter notebook 파일에 작성해주세요.
- 1번 문제는 정답 코드를 그대로 공개해드립니다. 따라서 진행해주세요.
- 그 외 문제들은 참고하실 수 있도록 주석과 출력 코드를 보여드립니다.
- 제출시, 반드시 결과가 실행되어 있어야 합니다. 미실행 또는 오류시 감점입니다.
- 제출시, [DS]sql3_choyongha.ipynb 형식에 맞춰 제출해주세요.


---

여러분은 이전 과제에서 세계 테러 데이터 분석 프로젝트를 진행하였습니다.

이번 시간에는 분석할 데이터를 Database 로 변환하고, 분석에 필요한 데이터를 저장된 Database 에서 쿼리하여 가져와 보도록 하겠습니다.

#### 작업환경

- aws rds 사용시: 이전 과제에서 생성한 AWS RDS (MySQL) 의 Database 및 사용자를 사용하세요.
  - Database Name: oneday
  - User Name / Password: oneday / 1234
- local 사용시: 로컬 환경에서 생성한 Database 및 사용자를 사용하세요.
  - 위와 동일하게 생성해주세요

## 원본 Data Download
- https://www.kaggle.com/datasets/START-UMD/gtd

---

문제 1.  csv 파일에 저장된 세계 테러 데이터를 하나의 테이블에 저장하세요.

- globalterrorismmdb_0718.csv - https://www.kaggle.com/datasets/START-UMD/gtd
- pandas의 to_sql 함수 사용 (힌트 : create_engine)
- Table name : origin_terror_data

제출 1. Table 생성 및 조회 결과 : DESC origin_terror_data, SELECT  COUNT(*) FROM origin_terror_data

제출 2. 관련 코드 및 실행 결과 (ipynb)



In [1]:
##############################################################################################
#  문제 1. 원본 데이터 (csv)를 데이터베이스로 변환하고 확인하기
#  globalterrorismdb_0718dist.csv -> origin_terror_data (table)
#  Pandas 의 to_sql 사용
##############################################################################################

##############################################################################################
#  pip install pymysql
#  pip install SQLAlchemy Flask-SQLAlchemy
##############################################################################################

import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import configparser

# 데이터 불러오기
df = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')

# DB 접속 엔진 객체 생성
user = 'oneday'
password = '1234'
host = 'localhost'
port = 3306
database = 'oneday'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4')

# DB 테이블 명(생성될 테이블 이름)
table_name = "origin_terror_data"

# DB에 DataFrame 적재
df.to_sql(index = False,
          name = table_name,
          con = engine,
          if_exists = 'append',
          method = 'multi',
          chunksize = 10000)

/var/folders/d2/b7bnbh8d1mx5rjyb450xlt200000gn/T/ipykernel_45301/3903147790.py:19: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')


181691

In [2]:
from sqlalchemy import text

with engine.connect() as connection:
    result = connection.execute(text("SELECT COUNT(*) FROM origin_terror_data"))
    records = result.fetchall()

print(records)

[(181691,)]


문제 2. origin_terror_data 에서 region, country 관련 데이터는 code 와 txt (name) 속성으로 정의되어 있습니다.

문제 2-1. Region 및 Country 테이블을 그림과 같은 구조로 생성하세요.

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요.
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요.
- Region 과 Country 데이터 사이의 관계를 파악하여 Foreign Key 를 설정하세요.

참고>
- Region.region_code     = origin_terror_data.region
- Region.region_name     = origin_terror_data.region_txt
- Country.country_code   = origin_terror_data.country
- Country.country_name   = origin_terror_data.country_txt


In [3]:
##############################################################################################
#  문제 2. region / country / city 데이터 추출하여 데이터베이스로 변환하기
#  중복을 제거한 code - name 값
#  region - country - city 관계 정의
##############################################################################################
# import pymysql

remote = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'oneday',
    password = '1234',
    database='oneday'
)

cursor = remote.cursor()

In [4]:
# region_txt 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(region_txt)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print(item)


(27,)


In [5]:
# Region 테이블 만들기
create_table_query = """
        CREATE TABLE IF NOT EXISTS Region (
            region_code INT NOT NULL,
            region_name VARCHAR(32) NOT NULL,
            PRIMARY KEY (region_code)
        );
        """
cursor.execute(create_table_query)

# 테이블 구조 확인
describe_query = "DESCRIBE Region;"
cursor.execute(describe_query)

# 결과
table_structure = cursor.fetchall()
print(table_structure)

(('region_code', 'int', 'NO', 'PRI', None, ''), ('region_name', 'varchar(32)', 'NO', '', None, ''))


In [6]:
# country_txt 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(country_txt)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print(item)

(32,)


In [7]:
# Country 테이블 만들기
create_table_query = """
        CREATE TABLE Country (
            country_code INT NOT NULL,
            region_code INT NOT NULL,
            country_name VARCHAR(32) NOT NULL,
            PRIMARY KEY (country_code)
        );
        """
cursor.execute(create_table_query)

# 테이블 구조 확인
describe_query = "DESCRIBE Country;"
cursor.execute(describe_query)

# 결과
table_structure = cursor.fetchall()
print(table_structure)

(('country_code', 'int', 'NO', 'PRI', None, ''), ('region_code', 'int', 'NO', '', None, ''), ('country_name', 'varchar(32)', 'NO', '', None, ''))


문제 2-2. origin_terror_data 테이블에서 Region 및 Country 데이터를 추출하여 문제 2-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요.
- 데이터를 INSERT 할때 순서를 고민하세요.

제출 3. Table 생성 및 조회 결과 : DESC Region, DESC Country

제출 4. 관련 코드 및 실행 결과 (ipynb)

In [8]:
# Region 데이터 추출하기
query = """
        SELECT DISTINCT region, region_txt
        FROM origin_terror_data
        ORDER BY region;
        """
cursor.execute(query)
# 결과
region_list = cursor.fetchall()
print(region_list)

((1, 'North America'), (2, 'Central America & Caribbean'), (3, 'South America'), (4, 'East Asia'), (5, 'Southeast Asia'), (6, 'South Asia'), (7, 'Central Asia'), (8, 'Western Europe'), (9, 'Eastern Europe'), (10, 'Middle East & North Africa'), (11, 'Sub-Saharan Africa'), (12, 'Australasia & Oceania'))


In [9]:
# Region 테이블에 INSERT
for region_id, region_name in region_list:
    query = f"""
            INSERT IGNORE INTO Region (region_code, region_name)
            VALUES({region_id}, '{region_name}');
            """
    cursor.execute(query)
remote.commit()

query = 'SELECT COUNT(*) FROM Region;'
cursor.execute(query)
print(cursor.fetchone())

(12,)


In [10]:
# Country 데이터 추출하기
query = """
        SELECT DISTINCT country, region, country_txt
        FROM origin_terror_data
        ORDER BY country_txt;
        """
cursor.execute(query)
# 결과
country_list = cursor.fetchall()
print(country_list)

((4, 6, 'Afghanistan'), (5, 9, 'Albania'), (6, 10, 'Algeria'), (7, 8, 'Andorra'), (8, 11, 'Angola'), (10, 2, 'Antigua and Barbuda'), (11, 3, 'Argentina'), (12, 7, 'Armenia'), (14, 12, 'Australia'), (15, 8, 'Austria'), (16, 7, 'Azerbaijan'), (17, 2, 'Bahamas'), (18, 10, 'Bahrain'), (19, 6, 'Bangladesh'), (20, 2, 'Barbados'), (35, 9, 'Belarus'), (21, 8, 'Belgium'), (22, 2, 'Belize'), (23, 11, 'Benin'), (25, 6, 'Bhutan'), (26, 3, 'Bolivia'), (28, 9, 'Bosnia-Herzegovina'), (29, 11, 'Botswana'), (30, 3, 'Brazil'), (31, 5, 'Brunei'), (32, 9, 'Bulgaria'), (33, 11, 'Burkina Faso'), (34, 11, 'Burundi'), (36, 5, 'Cambodia'), (37, 11, 'Cameroon'), (38, 1, 'Canada'), (41, 11, 'Central African Republic'), (42, 11, 'Chad'), (43, 3, 'Chile'), (44, 4, 'China'), (45, 3, 'Colombia'), (46, 11, 'Comoros'), (49, 2, 'Costa Rica'), (50, 9, 'Croatia'), (51, 2, 'Cuba'), (53, 8, 'Cyprus'), (54, 9, 'Czech Republic'), (236, 9, 'Czechoslovakia'), (229, 11, 'Democratic Republic of the Congo'), (55, 8, 'Denmark'), (

In [11]:
# Country 테이블에 INSERT
for cntry_id, region_id, cntry_name in country_list:
    query = f"""
            INSERT IGNORE INTO Country
            VALUES({cntry_id}, {region_id}, "{cntry_name}");
            """
    cursor.execute(query)
remote.commit()

query = 'SELECT * FROM Country;'
cursor.execute(query)
for item in cursor:
    print(item)

(4, 6, 'Afghanistan')
(5, 9, 'Albania')
(6, 10, 'Algeria')
(7, 8, 'Andorra')
(8, 11, 'Angola')
(10, 2, 'Antigua and Barbuda')
(11, 3, 'Argentina')
(12, 7, 'Armenia')
(14, 12, 'Australia')
(15, 8, 'Austria')
(16, 7, 'Azerbaijan')
(17, 2, 'Bahamas')
(18, 10, 'Bahrain')
(19, 6, 'Bangladesh')
(20, 2, 'Barbados')
(21, 8, 'Belgium')
(22, 2, 'Belize')
(23, 11, 'Benin')
(25, 6, 'Bhutan')
(26, 3, 'Bolivia')
(28, 9, 'Bosnia-Herzegovina')
(29, 11, 'Botswana')
(30, 3, 'Brazil')
(31, 5, 'Brunei')
(32, 9, 'Bulgaria')
(33, 11, 'Burkina Faso')
(34, 11, 'Burundi')
(35, 9, 'Belarus')
(36, 5, 'Cambodia')
(37, 11, 'Cameroon')
(38, 1, 'Canada')
(41, 11, 'Central African Republic')
(42, 11, 'Chad')
(43, 3, 'Chile')
(44, 4, 'China')
(45, 3, 'Colombia')
(46, 11, 'Comoros')
(47, 11, 'Republic of the Congo')
(49, 2, 'Costa Rica')
(50, 9, 'Croatia')
(51, 2, 'Cuba')
(53, 8, 'Cyprus')
(54, 9, 'Czech Republic')
(55, 8, 'Denmark')
(56, 11, 'Djibouti')
(57, 2, 'Dominica')
(58, 2, 'Dominican Republic')
(59, 3, 'Ecuado

문제 3.  origin_terror_data 에서 attack type, target type, weapon type 관련 데이터는 code 와 txt 속성으로 정의되어 있습니다.  

문제 3-1. AttackType, TargetType, WeaponType 테이블을 그림과 같은 구조로 생성하세요.

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요.
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요.

참고>
- AttackType.attacktype_code    = origin_terror_data.attacktype1
- AttackType.attacktype_desc    = origin_terror_data.attacktype1_txt
- TargetType.targtype_code      = origin_terror_data.targtype1
- TargetType.targtype_desc      = origin_terror_data.targtype1_txt
- WeaponType.weaptype_code      = origin_terror_data.weaptype1
- WeaponType.weaptype_desc      = origin_terror_data.weaptype1_txt

In [12]:
# attacktype1 의 max length 체크
# attacktype1은 int형이어서 max length는 1이고, attacktype1_txt를 체크해야 함.
cursor.execute('SELECT MAX(CHAR_LENGTH(attacktype1_txt)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print(item)

(35,)


In [13]:
# AttackType 테이블 만들기
create_table_query = """
        CREATE TABLE AttackType (
            attacktype_code INT NOT NULL,
            attacktype_desc VARCHAR(35) NOT NULL,
            PRIMARY KEY (attacktype_code)
        );
        """
cursor.execute(create_table_query)

# 테이블 구조 확인
describe_query = "DESCRIBE AttackType;"
cursor.execute(describe_query)

# 결과
table_structure = cursor.fetchall()
print(table_structure)

(('attacktype_code', 'int', 'NO', 'PRI', None, ''), ('attacktype_desc', 'varchar(35)', 'NO', '', None, ''))


In [14]:
# targettype1 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(targtype1_txt)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print(item)

(30,)


In [15]:
# TargetType 테이블 만들기
create_table_query = """
        CREATE TABLE TargetType (
            targtype_code INT NOT NULL,
            targtype_desc VARCHAR(32) NOT NULL,
            PRIMARY KEY (targtype_code)
        );
        """
cursor.execute(create_table_query)

# 테이블 구조 확인
describe_query = "DESCRIBE TargetType;"
cursor.execute(describe_query)

# 결과
table_structure = cursor.fetchall()
print(table_structure)

(('targtype_code', 'int', 'NO', 'PRI', None, ''), ('targtype_desc', 'varchar(32)', 'NO', '', None, ''))


In [16]:
# weaptype1 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(weaptype1_txt)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print(item)

(75,)


In [17]:
# WeaponType 테이블 만들기
create_table_query = """
        CREATE TABLE WeaponType (
            weaptype_code INT NOT NULL,
            weaptype_desc VARCHAR(80) NOT NULL,
            PRIMARY KEY (weaptype_code)
        );
        """
cursor.execute(create_table_query)

# 테이블 구조 확인
describe_query = "DESCRIBE WeaponType;"
cursor.execute(describe_query)

# 결과
table_structure = cursor.fetchall()
print(table_structure)

(('weaptype_code', 'int', 'NO', 'PRI', None, ''), ('weaptype_desc', 'varchar(80)', 'NO', '', None, ''))


문제 3-2. origin_terror_data 테이블에서 Attack Type, Target Type, Weapon Type 데이터를 추출하여 문제 3-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요.
- 데이터를 INSERT 할때 순서를 고민하세요.

제출 5. Table 생성 및 조회 결과 : DESC AttackType, DESC TargetType, WeaponType

제출 6. 관련 코드 및 실행 결과 (ipynb)

In [18]:
# attacktype1 데이터 추출하기
query = """
        SELECT DISTINCT attacktype1, attacktype1_txt
        FROM origin_terror_data
        ORDER BY attacktype1;
        """
cursor.execute(query)
# 결과
attack_list = cursor.fetchall()
print(attack_list)

((1, 'Assassination'), (2, 'Armed Assault'), (3, 'Bombing/Explosion'), (4, 'Hijacking'), (5, 'Hostage Taking (Barricade Incident)'), (6, 'Hostage Taking (Kidnapping)'), (7, 'Facility/Infrastructure Attack'), (8, 'Unarmed Assault'), (9, 'Unknown'))


In [19]:
# AttackType 테이블에 INSERT
for attack_id, attack_name in attack_list:
    query = f"""
            INSERT IGNORE INTO AttackType
            VALUES({attack_id},  "{attack_name}");
            """
    cursor.execute(query)
remote.commit()

query = 'SELECT * FROM AttackType;'
cursor.execute(query)
for item in cursor:
    print(item)

(1, 'Assassination')
(2, 'Armed Assault')
(3, 'Bombing/Explosion')
(4, 'Hijacking')
(5, 'Hostage Taking (Barricade Incident)')
(6, 'Hostage Taking (Kidnapping)')
(7, 'Facility/Infrastructure Attack')
(8, 'Unarmed Assault')
(9, 'Unknown')


In [20]:
# targtype1 데이터 추출하기
query = """
        SELECT DISTINCT targtype1, targtype1_txt
        FROM origin_terror_data
        ORDER BY targtype1;
        """
cursor.execute(query)
# 결과
target_list = cursor.fetchall()
print(target_list)

((1, 'Business'), (2, 'Government (General)'), (3, 'Police'), (4, 'Military'), (5, 'Abortion Related'), (6, 'Airports & Aircraft'), (7, 'Government (Diplomatic)'), (8, 'Educational Institution'), (9, 'Food or Water Supply'), (10, 'Journalists & Media'), (11, 'Maritime'), (12, 'NGO'), (13, 'Other'), (14, 'Private Citizens & Property'), (15, 'Religious Figures/Institutions'), (16, 'Telecommunication'), (17, 'Terrorists/Non-State Militia'), (18, 'Tourists'), (19, 'Transportation'), (20, 'Unknown'), (21, 'Utilities'), (22, 'Violent Political Party'))


In [21]:
# TargetType 테이블에 INSERT
for target_id, target_name in target_list:
    query = f"""
            INSERT IGNORE INTO TargetType
            VALUES({target_id},  "{target_name}");
            """
    cursor.execute(query)
remote.commit()

query = 'SELECT * FROM TargetType;'
cursor.execute(query)
for item in cursor:
    print(item)
# ALTER TABLE tablename
# CHANGE COLUMN old_columnname new_columnname new_datatype;

(1, 'Business')
(2, 'Government (General)')
(3, 'Police')
(4, 'Military')
(5, 'Abortion Related')
(6, 'Airports & Aircraft')
(7, 'Government (Diplomatic)')
(8, 'Educational Institution')
(9, 'Food or Water Supply')
(10, 'Journalists & Media')
(11, 'Maritime')
(12, 'NGO')
(13, 'Other')
(14, 'Private Citizens & Property')
(15, 'Religious Figures/Institutions')
(16, 'Telecommunication')
(17, 'Terrorists/Non-State Militia')
(18, 'Tourists')
(19, 'Transportation')
(20, 'Unknown')
(21, 'Utilities')
(22, 'Violent Political Party')


In [22]:
# weaptype1 데이터 추출하기
query = """
        SELECT DISTINCT weaptype1, weaptype1_txt
        FROM origin_terror_data
        ORDER BY weaptype1;
        """
cursor.execute(query)
# 결과
weapon_list = cursor.fetchall()
print(weapon_list)

((1, 'Biological'), (2, 'Chemical'), (3, 'Radiological'), (5, 'Firearms'), (6, 'Explosives'), (7, 'Fake Weapons'), (8, 'Incendiary'), (9, 'Melee'), (10, 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)'), (11, 'Sabotage Equipment'), (12, 'Other'), (13, 'Unknown'))


In [23]:
# WeaponType 테이블에 INSERT
for weapon_id, weapon_name in weapon_list:
    query = f"""
            INSERT IGNORE INTO WeaponType
            VALUES({weapon_id},  "{weapon_name}");
            """
    cursor.execute(query)
remote.commit()

query = 'SELECT * FROM WeaponType;'
cursor.execute(query)
for item in cursor:
    print(item)

(1, 'Biological')
(2, 'Chemical')
(3, 'Radiological')
(5, 'Firearms')
(6, 'Explosives')
(7, 'Fake Weapons')
(8, 'Incendiary')
(9, 'Melee')
(10, 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)')
(11, 'Sabotage Equipment')
(12, 'Other')
(13, 'Unknown')


문제 4. TerrorData 테이블을 만들고 앞서 만들어둔 테이블과 관계를 설정하도록 하겠습니다.
문제 4-1. TerrorData 테이블을 앞의 그림과 같이 생성하세요.

- origin_terror_data 를 분석하여 데이터 타입을 정의하세요.
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요.

참고>
- TerrorData.terror id : Auto Increment
- TerrorData.city_name    = origin_terror_data.city
- TerrorData.target       = orgin_terror_data.target1
- TerrorData.group_name   = origin_terror_data.gname
- TerrorData.kill_count   = origin_terror_data.nkill
- TerrorData.wound_count  = origin_terror_data.nwound
- TerrorData.motive       = origin_terror_data.motive
- TerrorData.summary      = origin_terror_data.summary
- TerrorData.latitude     = origin_terror_data_latitude
- TerrorData.longitude    = origin_terror_data_longitude
- TerrorData.terror_date  = origin_terror_data.iyear + origin_terror_Data.imonth + origin_terror_data.iday (Date Type)

In [24]:
# max length 체크

# city 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(city)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print('city:', item[0])

# target1 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(target1)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print('target1:', item[0])

# gname 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(gname)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print('gname:', item[0])

# summary 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(summary)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print('summary:', item[0])

# motive 의 max length 체크
cursor.execute('SELECT MAX(CHAR_LENGTH(motive)) AS max_length FROM origin_terror_data;')
for item in cursor:
    print('motive:', item[0])


city: 65
target1: 343
gname: 112
summary: 2431
motive: 899


In [25]:
# TerrorData 테이블 만들기
create_table_query = """
        CREATE TABLE TerrorData (
            terror_id INT NOT NULL AUTO_INCREMENT,
            terror_date DATE NOT NULL,
            region_code INT NOT NULL,
            country_code INT NOT NULL,
            city VARCHAR(65) NULL,
            target VARCHAR(350) NULL,
            group_name VARCHAR(120) NULL,
            targtype_code INT NOT NULL,
            attacktype_code INT NOT NULL,
            weaptype_code INT NOT NULL,
            kill_count INT NOT NULL DEFAULT 0,
            wound_count INT NOT NULL DEFAULT 0,
            motive VARCHAR(900) NULL,
            summary VARCHAR(2450) NULL,
            latitude DECIMAL(16,14) NULL,
            longitude DECIMAL(17,14) NULL,
            PRIMARY KEY (terror_id)
        );
        """
cursor.execute(create_table_query)

# 테이블 구조 확인
describe_query = "DESCRIBE TerrorData;"
cursor.execute(describe_query)

# 결과
table_structure = cursor.fetchall()
print(table_structure)

(('terror_id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('terror_date', 'date', 'NO', '', None, ''), ('region_code', 'int', 'NO', '', None, ''), ('country_code', 'int', 'NO', '', None, ''), ('city', 'varchar(65)', 'YES', '', None, ''), ('target', 'varchar(350)', 'YES', '', None, ''), ('group_name', 'varchar(120)', 'YES', '', None, ''), ('targtype_code', 'int', 'NO', '', None, ''), ('attacktype_code', 'int', 'NO', '', None, ''), ('weaptype_code', 'int', 'NO', '', None, ''), ('kill_count', 'int', 'NO', '', '0', ''), ('wound_count', 'int', 'NO', '', '0', ''), ('motive', 'varchar(900)', 'YES', '', None, ''), ('summary', 'varchar(2450)', 'YES', '', None, ''), ('latitude', 'decimal(16,14)', 'YES', '', None, ''), ('longitude', 'decimal(17,14)', 'YES', '', None, ''))


문제 4-2. origin_terror_data 테이블에서 Terror Data를 추출하여 문제 4-1.에서 생성한 테이블에 입력하고 확인하세요.

- 앞서 생성한 Region, Country, AttackType, TargetType, WeaponType 데이터와의 관계에 주의하세요.
- Count 값을 가지는 칼럼의 값이 null 인 경우, 0으로 예외처리 해주세요.
- 위도 경도 데이터 중 범위를 넘어서는 데이터가 존재합니다. 이 경우, null 값으로 예외처리 해주세요. (위도 경도 범위 : 구글링해보세요.)
- terror_date 칼럼의 경우, origin_terror_data 의 연, 월, 일 정보를 조합하여 date type 으로 정의해주세요. (Format : ‘YYYY-mm-dd’)
- origin_terror_data 의 월, 일 정보중 값이 0 인 경우 date type 으로 변환되지 않습니다. 이 경우, 1 로 예외처리 해주세요.
- 데이터 입력까지 완료한 이후, origin_terror_data 테이블을 삭제하고 확인하세요.

제출 7. Table 생성 및 조회 결과 : DESC TerrorData, SELECT COUNT(*) FROM TerrorData

제출 8. 관련 코드 및 실행 결과 (ipynb)

In [26]:
# data 전처리
# date type : year + month + day (month = 0 인경우 1, day = 0 인경우 1)
# nkill, nwound : null 인 경우 0
# longitude range : 180 ~ -180
# check : select longitude from origin_terror_data where longitude < -180 or longitude > 180; '-86185896'

query = """
SELECT 
    CONCAT_WS('-', 
        iyear,
        IF(imonth = 0, 1, imonth),
        IF(iday = 0, 1, iday)
    ) AS terror_date,
    region,
    country,
    city,
    target1,
    gname,
    targtype1,
    attacktype1,
    weaptype1,
    COALESCE(nkill, 0) AS kill_count,
    COALESCE(nwound, 0) AS wound_count,
    motive,
    summary,
    CASE 
        WHEN latitude BETWEEN -90 AND 90 THEN latitude
        ELSE NULL
    END AS latitude,
    CASE 
        WHEN longitude BETWEEN -180 AND 180 THEN longitude
        ELSE NULL
    END AS longitude
FROM origin_terror_data;
"""
cursor.execute(query)

# terror 데이터 추출하기
data_list = cursor.fetchall()
for item in data_list[1290:1292]:
    print(item)


('1972-5-19', 1, 217, 'Placentia', 'Valencia High School, Placentia California', 'Chicano Liberation Front', 8, 7, 8, 0.0, 0.0, None, '5/19/1971: Suspected members of the Chicano Liberation Front threw two Molotov cocktails at Valencia High School in Placentia, California, United States.  One firebomb crashed through a window of a secretarial office  and the other bounced off the building and ignited on the sidewalk.  There were no casualties but the incendiary devices caused an estimated $1,000 in damages.', 33.876161, -117.869263)
('1972-5-20', 8, 603, 'Unknown', 'Henry Gillespie, 32, Protestant', 'Irish Republican Army (IRA)', 4, 1, 5, 1.0, 0.0, None, None, 54.787715, -6.492314)


In [27]:
# 데이터 추출
query = """
SELECT 
    CONCAT_WS('-', 
        iyear,
        IF(imonth = 0, 1, imonth),
        IF(iday = 0, 1, iday)
    ) AS terror_date,
    region AS region_code,
    country AS country_code,
    city,
    target1 AS target,
    gname AS group_name,
    targtype1 AS targtype_code,
    attacktype1 AS attacktype_code,
    weaptype1 AS weaptype_code,
    COALESCE(nkill, 0) AS kill_count,
    COALESCE(nwound, 0) AS wound_count,
    motive,
    summary,
    CASE 
        WHEN latitude BETWEEN -90 AND 90 THEN latitude
        ELSE NULL
    END AS latitude,
    CASE 
        WHEN longitude BETWEEN -180 AND 180 THEN longitude
        ELSE NULL
    END AS longitude
FROM origin_terror_data;
"""
# engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4')

df = pd.read_sql(query, engine)

# 데이터 적재
df.to_sql(name='TerrorData', con=engine, if_exists='append', index=False)

181691

In [33]:
# 데이터 확인
remote.commit()
cursor.execute("SELECT COUNT(*) FROM TerrorData;")
result = cursor.fetchall()
print(result)

((181691,),)


In [34]:
# 테이블 삭제
cursor.execute("DROP TABLE IF EXISTS origin_terror_data;")

0

문제 5. TerrorData 의 전체 기간에서 테러의 숫자를 연도별로 집계하여 연도별 테러 발생 건수를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다.

제출 9. 관련 코드 및 실행 결과 (ipynb)

In [35]:
# Foreing Key 제약 설정 (테이블 생성 시 하면 더 좋았을텐데...)
query = """
        ALTER TABLE TerrorData
        ADD CONSTRAINT FK_region
        FOREIGN KEY (region_code)
        REFERENCES Region (region_code);        
        """
cursor.execute(query)

query = """
        ALTER TABLE TerrorData
        ADD CONSTRAINT FK_country
        FOREIGN KEY (country_code)
        REFERENCES Country (country_code);        
        """
cursor.execute(query)

query = """
        ALTER TABLE TerrorData
        ADD CONSTRAINT FK_attack
        FOREIGN KEY (attacktype_code)
        REFERENCES AttackType (attacktype_code);        
        """
cursor.execute(query)

query = """
        ALTER TABLE TerrorData
        ADD CONSTRAINT FK_weapon
        FOREIGN KEY (weaptype_code)
        REFERENCES WeaponType (weaptype_code);        
        """
cursor.execute(query)

query = """
        ALTER TABLE TerrorData
        ADD CONSTRAINT FK_target
        FOREIGN KEY (targtype_code)
        REFERENCES TargetType (targtype_code);        
        """
cursor.execute(query)

181691

In [37]:
query = """
        SELECT YEAR(terror_date) AS year, count(terror_id)
        FROM TerrorData
        GROUP BY year
        ORDER BY year;
        """
cursor.execute(query)

for item in cursor:
    print(item)

(1970, 651)
(1971, 471)
(1972, 568)
(1973, 473)
(1974, 581)
(1975, 740)
(1976, 923)
(1977, 1319)
(1978, 1526)
(1979, 2662)
(1980, 2662)
(1981, 2586)
(1982, 2544)
(1983, 2870)
(1984, 3495)
(1985, 2915)
(1986, 2860)
(1987, 3183)
(1988, 3721)
(1989, 4324)
(1990, 3887)
(1991, 4683)
(1992, 5071)
(1994, 3456)
(1995, 3081)
(1996, 3058)
(1997, 3197)
(1998, 934)
(1999, 1395)
(2000, 1814)
(2001, 1906)
(2002, 1333)
(2003, 1278)
(2004, 1166)
(2005, 2017)
(2006, 2758)
(2007, 3242)
(2008, 4805)
(2009, 4721)
(2010, 4826)
(2011, 5076)
(2012, 8522)
(2013, 12036)
(2014, 16903)
(2015, 14965)
(2016, 13587)
(2017, 10900)


문제 6. TerrorData 에서 테러가 가장 많이 일어난 순서로 국가를 정렬하여 상위 10위 국가를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다.
- 국가 정보는 이름으로 출력하세요.

제출 10. 관련 코드 및 실행 결과 (ipynb)

In [40]:
# 테러가 많이 일어난 상위 10위 Region
query = """
        SELECT rg.region_name, count(td.terror_id) as number
        FROM TerrorData td LEFT JOIN Region rg
        ON rg.region_code = td.region_code
        GROUP BY td.region_code
        ORDER BY number DESC
        Limit 10;
        """
cursor.execute(query)
print("테러가 많이 일어난 상위 10위 Region")
for item in cursor:
    print(item)

# 테러가 많이 일어난 상위 10위 Country
query = """
        SELECT ct.country_name, count(td.terror_id) as number
        FROM TerrorData td LEFT JOIN Country ct
        ON ct.country_code = td.country_code
        GROUP BY td.country_code
        ORDER BY number DESC
        Limit 10;
        """
cursor.execute(query)
print("테러가 많이 일어난 상위 10위 Country")
for item in cursor:
    print(item)


테러가 많이 일어난 상위 10위 Region
('Middle East & North Africa', 50474)
('South Asia', 44974)
('South America', 18978)
('Sub-Saharan Africa', 17550)
('Western Europe', 16639)
('Southeast Asia', 12485)
('Central America & Caribbean', 10344)
('Eastern Europe', 5144)
('North America', 3456)
('East Asia', 802)
테러가 많이 일어난 상위 10위 Country
('Iraq', 24636)
('Pakistan', 14368)
('Afghanistan', 12731)
('India', 11960)
('Colombia', 8306)
('Philippines', 6908)
('Peru', 6096)
('El Salvador', 5320)
('United Kingdom', 5235)
('Turkey', 4292)


문제 7. TerrorData 에서 테러가 가장 많이 일어난 상위 10위 국가에 대해 국가별로 사망자수와 부상자수, 사상자수(사망자수 + 부상자수)를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다.
- 국가 정보는 이름으로 출력하세요.

제출 11. 관련 코드 및 실행 결과 (ipynb)

In [61]:
query = """
        SELECT c.country_name, COUNT(*), SUM(t.kill_count) AS total_kill, 
                SUM(t.wound_count) AS total_wound, 
                SUM(t.kill_count + t.wound_count) AS total_casaulity 
        FROM TerrorData t 
        JOIN Country c ON t.country_code=c.country_code
        GROUP BY t.country_code 
        ORDER BY count(*) DESC 
        LIMIT 10;
        """
cursor.execute(query)
for item in cursor:
    print(item)

('Iraq', 24636, Decimal('78589'), Decimal('134690'), Decimal('213279'))
('Pakistan', 14368, Decimal('23822'), Decimal('42038'), Decimal('65860'))
('Afghanistan', 12731, Decimal('39384'), Decimal('44277'), Decimal('83661'))
('India', 11960, Decimal('19341'), Decimal('28980'), Decimal('48321'))
('Colombia', 8306, Decimal('14698'), Decimal('10328'), Decimal('25026'))
('Philippines', 6908, Decimal('9559'), Decimal('13367'), Decimal('22926'))
('Peru', 6096, Decimal('12771'), Decimal('4078'), Decimal('16849'))
('El Salvador', 5320, Decimal('12053'), Decimal('5062'), Decimal('17115'))
('United Kingdom', 5235, Decimal('3410'), Decimal('6106'), Decimal('9516'))
('Turkey', 4292, Decimal('6888'), Decimal('9899'), Decimal('16787'))


문제 8. 지역별 테러 공격 형태에 따른 사망자수, 부상자수, 사상자수를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다.
- 지역 정보(Region)는 이름으로 출력하세요.

제출 12. 관련 코드 및 실행 결과 (ipynb)

In [59]:
# 지역별 테러 공격 형태에 따른 사망자와 사상자 수
query = """
        SELECT
            r.region_name AS 지역,
            at.attacktype_desc AS 공격_형태,
            SUM(td.kill_count) AS 사망자수,
            SUM(td.kill_count + td.wound_count) AS 사상자수
        FROM
            TerrorData td
        JOIN
            Region r ON td.region_code = r.region_code
        JOIN
            AttackType at ON td.attacktype_code = at.attacktype_code
        GROUP BY
            r.region_name, at.attacktype_desc;
        """
cursor.execute(query)
for item in cursor:
    print(item)


('Central America & Caribbean', 'Assassination', Decimal('1699'), Decimal('1945'))
('Southeast Asia', 'Assassination', Decimal('1632'), Decimal('2453'))
('South America', 'Assassination', Decimal('4239'), Decimal('4894'))
('North America', 'Assassination', Decimal('238'), Decimal('322'))
('Middle East & North Africa', 'Assassination', Decimal('4774'), Decimal('9295'))
('South Asia', 'Assassination', Decimal('6701'), Decimal('11881'))
('Western Europe', 'Assassination', Decimal('2673'), Decimal('3474'))
('Sub-Saharan Africa', 'Assassination', Decimal('2371'), Decimal('3453'))
('Australasia & Oceania', 'Assassination', Decimal('34'), Decimal('43'))
('East Asia', 'Assassination', Decimal('44'), Decimal('79'))
('Eastern Europe', 'Assassination', Decimal('377'), Decimal('778'))
('Central Asia', 'Assassination', Decimal('138'), Decimal('190'))
('North America', 'Armed Assault', Decimal('837'), Decimal('2379'))
('South America', 'Armed Assault', Decimal('16814'), Decimal('20701'))
('Western E

---

# reference

민형동 (2020). Global Terrorism Data를 활용한 국가별 테러유형 분석. 한국공안행정학회, 82, 63-84.

한인택 (2017). Global Terrorism Database를 활용한 테러리즘의 현황분석.

민형기 (2019). 전세계 테러 데이터 분석 (따라하기). https://pinkwink.kr/1214

ASHWINI SWSAIN (2018). Terrorism Around The World. https://www.kaggle.com/code/ash316/terrorism-around-the-world/notebook

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021). *Global Terrorism Database* (GTD) [Data set]. University of Maryland. https://www.start.umd.edu/gtd

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021, August). *Global Terrorism Database codebook: Methodology, inclusion criteria, and variables.* University of Maryland. https://www.start.umd.edu/gtd/downloads/Codebook.pdf
